In [1]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

print(result)

42


In [2]:
# (p. 236) Linear regression with TensorFlow on the housing data set
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
# standard normal equation using TF API
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    
print(theta_value)

[[ -3.74651413e+01]
 [  4.35734153e-01]
 [  9.33829229e-03]
 [ -1.06622010e-01]
 [  6.44106984e-01]
 [ -4.25131839e-06]
 [ -3.77322501e-03]
 [ -4.26648885e-01]
 [ -4.40514028e-01]]


In [3]:
# (p. 237) Gradient descent with TF
n_epochs = 1000
learning_rate = 0.01

from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
scaled_housing_data_plus_bias = std_scaler.fit_transform(housing_data_plus_bias)

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, n], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
# this is one learning step:
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)    
    best_theta = theta.eval()
    print("Best theta:", best_theta)

Epoch 0 MSE = 7.82133
Epoch 100 MSE = 5.01968
Epoch 200 MSE = 4.93929
Epoch 300 MSE = 4.90486
Epoch 400 MSE = 4.88014
Epoch 500 MSE = 4.86177
Epoch 600 MSE = 4.848
Epoch 700 MSE = 4.83765
Epoch 800 MSE = 4.82983
Epoch 900 MSE = 4.82389
Best theta: [[ 0.69248843  0.49871492  0.3567431   0.52494383  0.49713802  0.37923646
  -0.91808748 -0.99268055]
 [ 0.89043927  0.95229042  0.8263191   0.79206502  0.84974414  0.92929751
   0.93021572  0.85590786]
 [ 0.16275936  0.17187308  0.16543671  0.15084441  0.1431676   0.17185833
   0.16035211  0.15066843]
 [-0.31945395 -0.44095641 -0.17081548 -0.1213785  -0.26493981 -0.38947713
  -0.41305661 -0.2647174 ]
 [ 0.32347026  0.42598498  0.18846822  0.15430821  0.28823116  0.37988055
   0.40898606  0.28282803]
 [ 0.01071668  0.01322949  0.01252289  0.00765906  0.00411785  0.01352151
   0.0093247   0.00676651]
 [-0.04434575 -0.04643553 -0.04320504 -0.0412403  -0.04182075 -0.04594377
  -0.04499023 -0.04258669]
 [-0.52196753 -0.39745292 -0.56266135 -0.7012

In [4]:
# (p. 238) same, but use TF for determining the gradients to make more efficient
n_epochs = 1000
learning_rate = 0.01

from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
scaled_housing_data_plus_bias = std_scaler.fit_transform(housing_data_plus_bias)

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, n], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")


# Here's the difference to the previous cell:
gradients = tf.gradients(mse, [theta])[0]

# this is one learning step:
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)    
    best_theta = theta.eval()
    print("Best theta:", best_theta)

Epoch 0 MSE = 9.28708
Epoch 100 MSE = 7.06094
Epoch 200 MSE = 6.04059
Epoch 300 MSE = 5.5443
Epoch 400 MSE = 5.28804
Epoch 500 MSE = 5.14795
Epoch 600 MSE = 5.06718
Epoch 700 MSE = 5.01828
Epoch 800 MSE = 4.98717
Epoch 900 MSE = 4.96636
Best theta: [[-0.27231669 -0.90357113  0.04143     0.56392193 -0.25176954 -0.70732045
   0.69734621 -0.60192037]
 [ 0.80211753  0.4431164   0.69785738  0.71045643  0.78773016  0.53916276
   0.59881037  0.76306242]
 [ 0.15972772  0.13483964  0.13153808  0.29945862  0.16280647  0.22520514
   0.03550499  0.28250521]
 [-0.02326218  0.40615213 -0.02726455  0.12314551 -0.00781702  0.44569701
   0.17828692  0.00450019]
 [-0.01581551 -0.27525666  0.03704283 -0.09446669 -0.00297341 -0.43076554
  -0.18938825  0.00494875]
 [ 0.0331336   0.00984912  0.01715174  0.13330343  0.03518312 -0.00957167
  -0.07889181  0.12449533]
 [-0.1060857  -0.03269896 -0.02890123 -0.0708595  -0.06339172 -0.03798709
  -0.03269131 -0.03094937]
 [ 0.02893702 -0.95684814 -0.11905522 -0.205

In [5]:
# (p. 239) same, but use TF optimizer
n_epochs = 1000
learning_rate = 0.01

from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
scaled_housing_data_plus_bias = std_scaler.fit_transform(housing_data_plus_bias)

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, n], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")


# Again, here's the difference to the previous cell:
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
# Alternative optimizer:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)    
    best_theta = theta.eval()
    print("Best theta:", best_theta)

Epoch 0 MSE = 9.01022
Epoch 100 MSE = 4.97823
Epoch 200 MSE = 4.91948
Epoch 300 MSE = 4.88258
Epoch 400 MSE = 4.85771
Epoch 500 MSE = 4.84088
Epoch 600 MSE = 4.82942
Epoch 700 MSE = 4.82158
Epoch 800 MSE = 4.81621
Epoch 900 MSE = 4.81247
Best theta: [[ 0.33016753  0.21522141  0.191998    0.54595566  0.54332042  0.65257764
   0.19866514 -0.48543692]
 [ 0.80725664  0.86026394  0.88439995  0.89806664  0.78299892  0.76336432
   0.85616076  0.85477597]
 [ 0.13288942  0.14734647  0.15290569  0.15727392  0.12503988  0.1342952
   0.13791645  0.15432644]
 [-0.18899082 -0.28068879 -0.32435626 -0.34679973 -0.14933462 -0.08804168
  -0.28913519 -0.25534061]
 [ 0.22732973  0.2994394   0.3346982   0.35177162  0.19725314  0.13592814
   0.3135477   0.27189103]
 [ 0.00093043  0.00550159  0.00721067  0.00862239 -0.00161029  0.00201153
   0.00212435  0.00811206]
 [-0.04004541 -0.04239857 -0.04338349 -0.0440444  -0.03886417 -0.03922845
  -0.04151338 -0.04287204]
 [-0.81782472 -0.66266209 -0.59947175 -0.554

In [13]:
# (p. 240) Batch feeding and TensorBoard
# (p. 242) TensorBoard stuff

# BEGIN logging for tensorboard
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%D%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)
# END logging for tensorboard

tf.reset_default_graph()

n_epochs = 2000
learning_rate = 0.01

import numpy.random as rnd

std_scaler = StandardScaler()
scaled_housing_data_plus_bias = std_scaler.fit_transform(housing_data_plus_bias)


X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()


# BEGIN logging for tensorboard
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
# END logging for tensorboard

init = tf.global_variables_initializer()
saver = tf.train.Saver()


def fetch_batch(epoch, batch_index, batch_size):
    rnd.seed(epoch * n_batches + batch_index)
    indices = rnd.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

n_epochs = 1000
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0: # save checkpoint
            save_path = saver.save(sess, '/tmp/my_model.ckpt')
            print("Epoch", epoch)
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()
    saver.save(sess, '/tmp/my_model_final.ckpt')
    
file_writer.close()
print("Best theta:")
print(best_theta)

Epoch 0
Epoch 100
Epoch 200
Epoch 300
Epoch 400
Epoch 500
Epoch 600
Epoch 700
Epoch 800
Epoch 900
Best theta:
[[ 0.90454292]
 [ 0.83778441]
 [ 0.10645497]
 [-0.25947902]
 [ 0.29196444]
 [ 0.00181689]
 [ 0.2128084 ]
 [-0.89034677]
 [-0.85242176]]
